# 🔍 Pesquisa - Básico",

Este notebook tem como objetivo apresentar **consultas básicas no Elasticsearch**.

Vamos explorar como realizar pesquisas simples utilizando a API de busca da ferramenta, com exemplos práticos e objetivos.

A ideia é mostrar como localizar documentos com base em seus campos, utilizando filtros e parâmetros comuns do dia a dia.

Para maiores, informações, acessar o [link](https://www.elastic.co/docs/solutions/search/full-text)